# Traing model in notebook
- start with a best model weight
- load and train it with different config
- save the new best model if miou increased

# Target
- best optimizer + lr + weight decay + lr_mult
- best batch size + data augmentation
- best loss function

In [1]:
import sys
import os
project_dir=os.path.expanduser('~/git/torchseg')
os.chdir(project_dir)
sys.path.insert(0,project_dir)
print(sys.path)
from utils.config import get_parser,get_config
from utils.disc_tools import get_newest_file
from models.pspnet import pspnet
import glob
import torch
%matplotlib inline
import matplotlib.pyplot as plt
from utils.survey import dataset_survey
from utils.metrics import runningScore,get_scores
from PIL import Image
from tqdm import tqdm

['/home/yzbx/git/torchseg', '', '/home/yzbx/git/torchseg/notebooks', '/home/yzbx/git/gnu/models/research', '/home/yzbx/git/gnu/models/research/slim', '/home/yzbx/bin/miniconda3/envs/new/lib/python36.zip', '/home/yzbx/bin/miniconda3/envs/new/lib/python3.6', '/home/yzbx/bin/miniconda3/envs/new/lib/python3.6/lib-dynload', '/home/yzbx/bin/miniconda3/envs/new/lib/python3.6/site-packages', '/home/yzbx/bin/miniconda3/envs/new/lib/python3.6/site-packages/cityscapesscripts-1.0.0-py3.6-linux-x86_64.egg', '/home/yzbx/bin/miniconda3/envs/new/lib/python3.6/site-packages/torchvision-0.2.1-py3.6.egg', '/home/yzbx/bin/miniconda3/envs/new/lib/python3.6/site-packages/IPython/extensions', '/home/yzbx/.ipython']


In [6]:
#checkpoint_path=os.path.expanduser('~/tmp/logs/pytorch/pspnet/voc2012/voc_benchmark/2018-10-26___17-00-45/model-last-49.pkl')
parser=get_parser()
argv='--upsample_layer=4 --use_momentum=True --backbone_name=vgg16_bn --use_lr_mult=True --changed_lr_mult=2 --new_lr_mult=5 --midnet_scale=5 --batch_size=4 --momentum=0.01 --dataset_name=VOC2012 --note=jupyter001 --save_model=True --n_epoch=50'
args=parser.parse_args(argv.split(' '))
config=get_config(args)
print(config)

model = globals()[args.net_name](config)

checkpoint_path=None
if checkpoint_path is None:
    log_dir = os.path.join(config.args.log_dir, model.name,
                       config.dataset.name, config.args.note)
    ckpt_files=glob.glob(os.path.join(log_dir,'**','model-best-*.pkl'),recursive=True)

    # use best model first, then use the last model, because the last model will be the newest one if exist.
    if len(ckpt_files)==0:
        ckpt_files=glob.glob(os.path.join(log_dir,'**','*.pkl'),recursive=True)

    assert len(ckpt_files)>0,'no weight file found under %s, \n please specify checkpoint path'%log_dir
    checkpoint_path=get_newest_file(ckpt_files)
    print('no checkpoint file given, auto find %s'%checkpoint_path)
else:
    assert os.path.exists(checkpoint_path),'checkpoint path %s not exist'%checkpoint_path
    
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

state_dict=torch.load(checkpoint_path)
if 'model_state' in state_dict.keys():
    model.load_state_dict(state_dict['model_state'])
else:
    model.load_state_dict(state_dict)
model.train()

convert input shape is (240, 240) ******************************
{'model': {'upsample_type': 'bilinear', 'auxnet_type': 'bilinear', 'upsample_layer': 4, 'auxnet_layer': 4, 'cross_merge_times': 1, 'use_momentum': True, 'backbone_pretrained': True, 'use_bn': True, 'use_dropout': False, 'use_bias': True, 'eps': 1e-05, 'momentum': 0.01, 'learning_rate': 0.0001, 'optimizer': 'adam', 'lr_weight_decay': 0.0001, 'lr_momentum': 0.9, 'use_lr_mult': True, 'changed_lr_mult': 2.0, 'new_lr_mult': 5.0, 'use_reg': False, 'use_class_weight': False, 'class_weight_alpha': 0.0, 'focal_loss_gamma': -1.0, 'focal_loss_alpha': 1.0, 'l2_reg': 1e-05, 'backbone_name': 'vgg16_bn', 'backbone_freeze': False, 'layer_preference': 'first', 'edge_seg_order': 'same', 'midnet_pool_sizes': [6, 3, 2, 1], 'midnet_scale': 5, 'midnet_name': 'psp', 'edge_bg_weight': 0.01, 'edge_base_weight': 1.0, 'edge_power': 0.9, 'aux_base_weight': 1.0, 'input_shape': [240, 240], 'midnet_out_channels': 512, 'class_number': 21}, 'dataset': {'

AssertionError: no weight file found under /home/yzbx/tmp/logs/pytorch/pspnet/voc2012/jupyter001, 
 please specify checkpoint path

# cycle lr mult

In [ ]:
n_epoch=15
net = model
config.args.note+='_cycle'
for times in range(3):
    config.args.n_epoch=n_epoch*2**times
    config.args.note = note+'_%d'%times
    assert net.config.args.n_epoch==config.args.n_epoch
    keras_fit(model=net,train_loader=train_loader,val_loader=val_loader)